### This notebook use for comparing 3 algorithm:
    - Graph Convolution Neural Network-Diachronic Embedding-SimplE
    - Diachronic Embedding-SimplE
    - Diachronic Embedding-TransE
## Structure:
    Support object:
        - Scripts
        - Params 
        - Measure 
    Machine learning format:
        - Dataset
        - tester
        - trainer
    Model (User can modify another Graph of anotehr Diachronic)
        - G_de_simple:Graph Convolution Neural Network-Diachronic Embedding-SimplE
        - G_simple: Graph Convolution Neural Network-SimplE
        - DE-simple: Diachronic Embedding-SimplE
        - DE-TransE: Diachronic Embedding-TransE
## How to use:
    1. Upload this notebook and data follow it to kaggle
    2. Change hyperparameter in block main.py
    3. Access to main.py to run or change hyperparameter

    A few notes:
        - Every model parameter save in "/kaggle/working/models "
        - Information about each run (include both train and test) save in "/kaggle/working/exps"
    





In [8]:
cd /kaggle/working/ # switch to kaggle/working to save model

/kaggle/working


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import math
import copy
import time
import numpy as np
from random import shuffle
from datetime import datetime
import time
import contextlib

# Scripts.py

In [12]:
def shredFacts(facts): #takes a batch of facts and shreds it into its columns
        
    heads      = torch.tensor(facts[:,0]).long().cuda()
    rels       = torch.tensor(facts[:,1]).long().cuda()
    tails      = torch.tensor(facts[:,2]).long().cuda()
    years = torch.tensor(facts[:,3]).float().cuda()
    months = torch.tensor(facts[:,4]).float().cuda()
    days = torch.tensor(facts[:,5]).float().cuda()
    return heads, rels, tails, years, months, days


# Params.py

In [13]:
class Params:

    def __init__(self, 
                 ne=500, 
                 bsize=512, 
                 lr=0.001, 
                 reg_lambda=0.0, 
                 emb_dim=100, 
                 neg_ratio=20, 
                 dropout=0.4,  
                 save_each=50,  
                 se_prop=0.9
                ):

        self.ne = ne
        self.bsize = bsize
        self.lr = lr
        self.reg_lambda = reg_lambda
        self.s_emb_dim = int(se_prop*emb_dim)
        self.t_emb_dim = emb_dim - int(se_prop*emb_dim)
        self.save_each = save_each
        self.neg_ratio = neg_ratio
        self.dropout = dropout
        self.se_prop = se_prop
        self.emb_dim = emb_dim
        
    def str_(self):
        return str(self.ne) + "_" + str(self.bsize) + "_" + str(self.lr) + "_" + str(self.reg_lambda) + "_" + str(self.s_emb_dim) + "_" + str(self.neg_ratio) + "_" + str(self.dropout) + "_" + str(self.t_emb_dim) + "_" + str(self.save_each) + "_" + str(self.se_prop) 

# Measure.py

In [14]:

class Measure:
    
    def __init__(self):
        self.hit1  = {"raw": 0.0, "fil": 0.0}
        self.hit3  = {"raw": 0.0, "fil": 0.0}
        self.hit10 = {"raw": 0.0, "fil": 0.0}
        self.mrr   = {"raw": 0.0, "fil": 0.0}
        self.mr    = {"raw": 0.0, "fil": 0.0}
        
    def update(self, rank, raw_or_fil):
        if rank == 1:
            self.hit1[raw_or_fil] += 1.0
        if rank <= 3:
            self.hit3[raw_or_fil] += 1.0
        if rank <= 10:
            self.hit10[raw_or_fil] += 1.0
            
        self.mr[raw_or_fil]  += rank
        self.mrr[raw_or_fil] += (1.0 / rank)
        
    def normalize(self, num_facts):
        for raw_or_fil in ["raw", "fil"]:
            self.hit1[raw_or_fil]  /= (2 * num_facts)
            self.hit3[raw_or_fil]  /= (2 * num_facts)
            self.hit10[raw_or_fil] /= (2 * num_facts)
            self.mr[raw_or_fil]    /= (2 * num_facts)
            self.mrr[raw_or_fil]   /= (2 * num_facts)
            
    def print_(self):

        for raw_or_fil in ["raw", "fil"]:
            print(raw_or_fil.title() + " setting:")
            print("\tHit@1 =",  self.hit1[raw_or_fil])
            print("\tHit@3 =",  self.hit3[raw_or_fil])
            print("\tHit@10 =", self.hit10[raw_or_fil])
            print("\tMR =",     self.mr[raw_or_fil])
            print("\tMRR =",    self.mrr[raw_or_fil])
            print("")
    def print__(self):
        for raw_or_fil in ["fil"]:
            print(raw_or_fil.title() + " setting:")
            print("")
            print("Hit@1 =",  self.hit1[raw_or_fil])
            print("Hit@3 =",  self.hit3[raw_or_fil])
            print("Hit@10 =", self.hit10[raw_or_fil])
            print("MR =",     self.mr[raw_or_fil])
            print("MRR =",    self.mrr[raw_or_fil])
            

# Dataset.py

In [15]:
class Dataset:
    """Implements the specified dataloader"""
    def __init__(self, ds_name):
        """
        Params: ds_name : name of the dataset 
        """
        self.name = ds_name
        self.ds_path = ds_name
        self.ent2id = {}
        self.rel2id = {}
        self.data = {"train": self.readFile(self.ds_path + "train.txt"),
                     "valid": self.readFile(self.ds_path + "valid.txt"),
                     "test":  self.readFile(self.ds_path + "test.txt")}
        
        self.start_batch = 0
        self.all_facts_as_tuples = None
        
        self.convertTimes()
        
        self.all_facts_as_tuples = set([tuple(d) for d in self.data["train"] + self.data["valid"] + self.data["test"]])
        
        for spl in ["train", "valid", "test"]:
            self.data[spl] = np.array(self.data[spl])
        
    def readFile(self, 
                 filename):
        #Input : path_file
        #Output: list facts
            #facts[0] = [1,2,3,2014-06-27]
      
        with open(filename, "r",encoding='utf-8') as f:
            data = f.readlines()
        
        facts = []
        for line in data:
            elements = line.strip().split("\t")
            
            head_id =  self.getEntID(elements[0])
            rel_id  =  self.getRelID(elements[1])
            tail_id =  self.getEntID(elements[2])
            timestamp = elements[3]
            
            facts.append([head_id, rel_id, tail_id, timestamp])
            
        return facts
    
    
    def convertTimes(self):      
        """
        This function spits the timestamp in the day,date and time.
        """  
        for split in ["train", "valid", "test"]:
            for i, fact in enumerate(self.data[split]):
                fact_date = fact[-1]
                self.data[split][i] = self.data[split][i][:-1]
                date = list(map(float, fact_date.split("-")))
                self.data[split][i] += date
        #Change data :
            # Exp: 2014-06-27 --> [2014,6,27]
                
                
    
    def numEnt(self): 
    
        return len(self.ent2id)

    def numRel(self): 
    
        return len(self.rel2id)

    
    def getEntID(self,ent_name):

        if ent_name in self.ent2id:
            return self.ent2id[ent_name] 
        self.ent2id[ent_name] = len(self.ent2id)
        return self.ent2id[ent_name]
    
    def getRelID(self, rel_name):
        if rel_name in self.rel2id:
            return self.rel2id[rel_name] 
        self.rel2id[rel_name] = len(self.rel2id)
        return self.rel2id[rel_name]

    
    def nextPosBatch(self, batch_size): 
        #Input: batch size
        #Output:
            # ret_facts.shape = [batch_size x features]
            # ret_facts[0] = [head,relation,tail,year,month,day] 
        if self.start_batch + batch_size > len(self.data["train"]):
            ret_facts = self.data["train"][self.start_batch : ]
            self.start_batch = 0
        else:
            ret_facts = self.data["train"][self.start_batch : self.start_batch + batch_size]
            self.start_batch += batch_size


        return ret_facts
    

    def addNegFacts(self, bp_facts, neg_ratio):
        ex_per_pos = 2 * neg_ratio + 2
        facts = np.repeat(np.copy(bp_facts), ex_per_pos, axis=0)
        for i in range(bp_facts.shape[0]):
            s1 = i * ex_per_pos + 1
            e1 = s1 + neg_ratio
            s2 = e1 + 1
            e2 = s2 + neg_ratio
            
            facts[s1:e1,0] = (facts[s1:e1,0] + np.random.randint(low=1, high=self.numEnt(), size=neg_ratio)) % self.numEnt()
            facts[s2:e2,2] = (facts[s2:e2,2] + np.random.randint(low=1, high=self.numEnt(), size=neg_ratio)) % self.numEnt()
            
        return facts
    
    def addNegFacts2(self, bp_facts, neg_ratio):
        # input bp_facts, neg_ratio
            # bp_facts.shape = [batch_size x features]
        # output A = concat(facts1,facts2)
        
            # A.shape = [batch_size x (neg_ratio + 1)] x [features]
            # A = [[1,2,3,4,5,6],
            #      . . . 
            #      [1,2,3,4,5,6]]
        
        pos_neg_group_size = 1 + neg_ratio
        facts1 = np.repeat(np.copy(bp_facts), pos_neg_group_size, axis=0)
        facts2 = np.copy(facts1)
        rand_nums1 = np.random.randint(low=1, high=self.numEnt(), size=facts1.shape[0])
        rand_nums2 = np.random.randint(low=1, high=self.numEnt(), size=facts2.shape[0])
        
        for i in range(facts1.shape[0] // pos_neg_group_size):
            rand_nums1[i * pos_neg_group_size] = 0
            rand_nums2[i * pos_neg_group_size] = 0
        
        facts1[:,0] = (facts1[:,0] + rand_nums1) % self.numEnt()
        facts2[:,2] = (facts2[:,2] + rand_nums2) % self.numEnt()

   
        return np.concatenate((facts1, facts2), axis=0)

  
    
    def nextBatch(self, batch_size, neg_ratio=1):
        bp_facts = self.nextPosBatch(batch_size)
        batch = shredFacts(self.addNegFacts2(bp_facts, neg_ratio)) #=>  batch = 6 tensors, tensor 1 = heads, tensor 2 = rels , . . . 
        alldata = torch.tensor(self.data['train']).float() # => alldata.shape = [number_instance x features]
       
        return  batch, alldata #batch = (heads, rels, tails, years, months, day) 
    
    
    def wasLastBatch(self):
        return (self.start_batch == 0)

# Graph representation 

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, node_features, adj):
      
        support = torch.mm(node_features, self.weight)  # [num_entities, out_features]
        output = torch.mm(adj, support)  # Aggretage function [num_entities, out_features]
        output = output + self.bias  # add bias
        return output


class GraphRepresentation(nn.Module):
    def __init__(self, num_entities, num_relations, t_emb_dim):
        super(GraphRepresentation, self).__init__()
        self.num_entities = num_entities
        self.num_relations = num_relations
        self.t_emb_dim = t_emb_dim

        self.entity_embeddings = nn.Embedding(num_entities, t_emb_dim)
        
        self.graph_conv1 = GraphConvolution(t_emb_dim, t_emb_dim)
        self.graph_conv2 = GraphConvolution(t_emb_dim, t_emb_dim)

    def forward(self, input_tensor, adj):

        heads = input_tensor[:, 0].long()  
        rels = input_tensor[:, 1].long()   
        tails = input_tensor[:, 2].long()  

        entity_features = self.entity_embeddings.weight  #  [num_entities, t_emb_dim]

        x = self.graph_conv1(entity_features, adj)  # [num_entities, t_emb_dim]
        x = F.relu(x)
        x = self.graph_conv2(x, adj)  # [num_entities, t_emb_dim]
        

        heads_embedding = x[heads]  #  [batch_size, t_emb_dim]
     
        return heads_embedding


# G_de_simple.py

In [21]:


class G_DE_SimplE(torch.nn.Module):
    def __init__(self, dataset, params):
        super(G_DE_SimplE, self).__init__()
        self.dataset = dataset
        self.params = params

        self.s_emb_dim = params.s_emb_dim
        self.g_emb_dim = params.s_emb_dim
        self.t_emb_dim = params.emb_dim - self.s_emb_dim - self.g_emb_dim
        
        self.ent_embs_h = nn.Embedding(dataset.numEnt(),  self.s_emb_dim).cuda()
        self.ent_embs_t = nn.Embedding(dataset.numEnt(),  self.s_emb_dim).cuda()
        self.rel_embs_f = nn.Embedding(dataset.numRel(), self.s_emb_dim + self.g_emb_dim + self.t_emb_dim).cuda()
        self.rel_embs_i = nn.Embedding(dataset.numRel(), self.s_emb_dim + self.g_emb_dim + self.t_emb_dim).cuda()
        
        self.create_time_embedds()

        self.graphRepresent = GraphRepresentation(dataset.numEnt(), dataset.numRel(), self.g_emb_dim)

        self.time_nl = torch.sin
        
        nn.init.xavier_uniform_(self.ent_embs_h.weight)
        nn.init.xavier_uniform_(self.ent_embs_t.weight)
        nn.init.xavier_uniform_(self.rel_embs_f.weight)
        nn.init.xavier_uniform_(self.rel_embs_i.weight)
    
    def create_time_embedds(self):

        # frequency embeddings for the entities
        self.m_freq_h = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.m_freq_t = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.d_freq_h = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.d_freq_t = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.y_freq_h = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.y_freq_t = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()

        # phi embeddings for the entities
        self.m_phi_h = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.m_phi_t = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.d_phi_h = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.d_phi_t = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.y_phi_h = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.y_phi_t = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()

        # frequency embeddings for the entities
        self.m_amps_h = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.m_amps_t = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.d_amps_h = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.d_amps_t = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.y_amps_h = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        self.y_amps_t = nn.Embedding(self.dataset.numEnt(), self.t_emb_dim).cuda()
        nn.init.xavier_uniform_(self.m_freq_h.weight)
        nn.init.xavier_uniform_(self.d_freq_h.weight)
        nn.init.xavier_uniform_(self.y_freq_h.weight)
        nn.init.xavier_uniform_(self.m_freq_t.weight)
        nn.init.xavier_uniform_(self.d_freq_t.weight)
        nn.init.xavier_uniform_(self.y_freq_t.weight)

        nn.init.xavier_uniform_(self.m_phi_h.weight)
        nn.init.xavier_uniform_(self.d_phi_h.weight)
        nn.init.xavier_uniform_(self.y_phi_h.weight)
        nn.init.xavier_uniform_(self.m_phi_t.weight)
        nn.init.xavier_uniform_(self.d_phi_t.weight)
        nn.init.xavier_uniform_(self.y_phi_t.weight)

        nn.init.xavier_uniform_(self.m_amps_h.weight)
        nn.init.xavier_uniform_(self.d_amps_h.weight)
        nn.init.xavier_uniform_(self.y_amps_h.weight)
        nn.init.xavier_uniform_(self.m_amps_t.weight)
        nn.init.xavier_uniform_(self.d_amps_t.weight)
        nn.init.xavier_uniform_(self.y_amps_t.weight)

    def get_time_embedd(self, entities, years, months, days, h_or_t):
        if h_or_t == "head":
            emb  = self.y_amps_h(entities) * self.time_nl(self.y_freq_h(entities) * years  + self.y_phi_h(entities))
            emb += self.m_amps_h(entities) * self.time_nl(self.m_freq_h(entities) * months + self.m_phi_h(entities))
            emb += self.d_amps_h(entities) * self.time_nl(self.d_freq_h(entities) * days   + self.d_phi_h(entities))
        else:
            emb  = self.y_amps_t(entities) * self.time_nl(self.y_freq_t(entities) * years  + self.y_phi_t(entities))
            emb += self.m_amps_t(entities) * self.time_nl(self.m_freq_t(entities) * months + self.m_phi_t(entities))
            emb += self.d_amps_t(entities) * self.time_nl(self.d_freq_t(entities) * days   + self.d_phi_t(entities))
            
        return emb
    def create_adjacency_matrix(self, input_tensor, num_entities):

        device = input_tensor.device
        
        heads = input_tensor[:, 0].long().to(device)  
        tails = input_tensor[:, 2].long().to(device) 
        indices = torch.stack([heads, tails], dim=0).to(device)  
    
        values = torch.ones(indices.shape[1], dtype=torch.float32).to(device)
    
        adj = torch.sparse_coo_tensor(indices, values, (num_entities, num_entities)).to(device)

        adj += torch.sparse_coo_tensor(
            torch.arange(num_entities).unsqueeze(0).repeat(2, 1).to(device), 
            torch.ones(num_entities, dtype=torch.float32).to(device), 
            (num_entities, num_entities)
        ).to(device)
    

        degree = torch.sparse.sum(adj, dim=1).to_dense().to(device)  
        degree_inv = 1.0 / degree
        degree_inv[degree == 0] = 0  
    
        values = values * degree_inv[heads].to(device)
        adj = torch.sparse_coo_tensor(indices, values, (num_entities, num_entities)).to(device)
    
        return adj

    def get_graph_embedd(self, heads, rels, tails, alldata):


        batch = torch.stack((heads, rels, tails), dim=1)
        alldata = alldata[:, :3] 
        combine_batch_alldata = torch.cat((batch,alldata),dim = 0) 

        start = time.time()
        adjmatrix = self.create_adjacency_matrix(combine_batch_alldata,self.dataset.numEnt())
        end = time.time()

        representGraph = self.graphRepresent(combine_batch_alldata, adjmatrix.cuda())

        return representGraph[:batch.shape[0]]
    

    def getEmbeddings(self, heads, rels, tails, years, months, days, alldata):
        years = years.view(-1,1)
        months = months.view(-1,1)
        days = days.view(-1,1)
        h_embs1 = self.ent_embs_h(heads)
        r_embs1 = self.rel_embs_f(rels)
        t_embs1 = self.ent_embs_t(tails)
        h_embs2 = self.ent_embs_h(tails)
        r_embs2 = self.rel_embs_i(rels)
        t_embs2 = self.ent_embs_t(heads)
        
        h_embs1 = torch.cat((h_embs1, self.get_time_embedd(heads, years, months, days, "head"),self.get_graph_embedd(heads, rels, tails, alldata)), 1)
        t_embs1 = torch.cat((t_embs1, self.get_time_embedd(tails, years, months, days, "tail"),self.get_graph_embedd(heads, rels, tails, alldata)), 1)
        h_embs2 = torch.cat((h_embs2, self.get_time_embedd(tails, years, months, days, "head"),self.get_graph_embedd(heads, rels, tails, alldata)), 1)
        t_embs2 = torch.cat((t_embs2, self.get_time_embedd(heads, years, months, days, "tail"),self.get_graph_embedd(heads, rels, tails, alldata)), 1)
        
        return h_embs1, r_embs1, t_embs1, h_embs2, r_embs2, t_embs2
    
    def forward(self, heads, rels, tails, years, months, days,alldata):
        h_embs1, r_embs1, t_embs1, h_embs2, r_embs2, t_embs2 = self.getEmbeddings(heads, rels, tails, years, months, days,alldata)
        scores = ((h_embs1 * r_embs1) * t_embs1 + (h_embs2 * r_embs2) * t_embs2) / 2.0
        scores = F.dropout(scores, p=self.params.dropout, training=self.training)
        scores = torch.sum(scores, dim=1)
        return scores
        


# G_simple.py

In [22]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

class G_SimplE(torch.nn.Module):
    def __init__(self, dataset, params):
        super(G_SimplE, self).__init__()
        self.dataset = dataset
        self.params = params

        self.ent_embs_h = nn.Embedding(dataset.numEnt(), params.s_emb_dim)
        self.ent_embs_t = nn.Embedding(dataset.numEnt(), params.s_emb_dim)
        self.rel_embs_f = nn.Embedding(dataset.numRel(), params.s_emb_dim + params.t_emb_dim)
        self.rel_embs_i = nn.Embedding(dataset.numRel(), params.s_emb_dim + params.t_emb_dim)

        nn.init.xavier_uniform_(self.ent_embs_h.weight)
        nn.init.xavier_uniform_(self.ent_embs_t.weight)
        nn.init.xavier_uniform_(self.rel_embs_f.weight)
        nn.init.xavier_uniform_(self.rel_embs_i.weight)

        self.graphRepresent = GraphRepresentation(dataset.numEnt(), dataset.numRel(), params.t_emb_dim)

    def getGraphBeforeTime(self, Current_Year, Current_Month, Current_Day, alldata): 
        alldata = alldata.cuda() if not alldata.is_cuda else alldata

        current_date = torch.tensor([Current_Year, Current_Month, Current_Day], device=alldata.device)

        data_dates = alldata[:, 3:]

        before_mask = (
            (data_dates[:, 0] < current_date[0]) |
            ((data_dates[:, 0] == current_date[0]) & (data_dates[:, 1] < current_date[1])) |
            ((data_dates[:, 0] == current_date[0]) & (data_dates[:, 1] == current_date[1]) & (data_dates[:, 2] < current_date[2]))
        )

        filtered_data = alldata[before_mask]

        return filtered_data


    def create_adjacency_matrix(self, input_tensor, num_entities):

        device = input_tensor.device
        
        heads = input_tensor[:, 0].long().to(device)  # Cột đầu tiên là heads
        tails = input_tensor[:, 2].long().to(device)  # Cột thứ ba là tails
        
        indices = torch.stack([heads, tails], dim=0).to(device)  
    
        values = torch.ones(indices.shape[1], dtype=torch.float32).to(device)
    
        adj = torch.sparse_coo_tensor(indices, values, (num_entities, num_entities)).to(device)
    
        adj += torch.sparse_coo_tensor(
            torch.arange(num_entities).unsqueeze(0).repeat(2, 1).to(device), 
            torch.ones(num_entities, dtype=torch.float32).to(device), 
            (num_entities, num_entities)
        ).to(device)
    
        degree = torch.sparse.sum(adj, dim=1).to_dense().to(device) 
        
        degree_inv = 1.0 / degree
        degree_inv[degree == 0] = 0 
        values = values * degree_inv[heads].to(device)
        adj = torch.sparse_coo_tensor(indices, values, (num_entities, num_entities)).to(device)
    
        return adj


    
    def get_graph_embedd(self, heads, rels, tails, years, months, days, alldata):


        batch = torch.stack((heads, rels, tails), dim=1) # get batch 
        alldata = alldata[:, :3] # get only head , realtion and tail
        combine_batch_alldata = torch.cat((batch,alldata),dim = 0) #  add bactch and alldata for Graph Representation
        adjmatrix = self.create_adjacency_matrix(combine_batch_alldata,self.dataset.numEnt())
        representGraph = self.graphRepresent(combine_batch_alldata, adjmatrix.cuda())
      
        return representGraph[:batch.shape[0]]


    

    def getEmbeddings(self, heads, rels, tails, years, months, days, alldata):
        years = years.view(-1, 1)
        months = months.view(-1, 1)
        days = days.view(-1, 1)

        h_embs1 = self.ent_embs_h(heads)
        r_embs1 = self.rel_embs_f(rels)
        t_embs1 = self.ent_embs_t(tails)
        h_embs2 = self.ent_embs_h(tails)
        r_embs2 = self.rel_embs_i(rels)
        t_embs2 = self.ent_embs_t(heads)


        h_embs1 = torch.cat((h_embs1, self.get_graph_embedd(heads, rels, tails, years, months, days, alldata)), 1)
        t_embs1 = torch.cat((t_embs1, self.get_graph_embedd(tails, rels, heads, years, months, days, alldata)), 1)
        h_embs2 = torch.cat((h_embs2, self.get_graph_embedd(tails, rels, heads, years, months, days, alldata)), 1)
        t_embs2 = torch.cat((t_embs2, self.get_graph_embedd(heads, rels, tails, years, months, days, alldata)), 1)

        return h_embs1, r_embs1, t_embs1, h_embs2, r_embs2, t_embs2

    def forward(self, heads, rels, tails, years, months, days, alldata):
        h_embs1, r_embs1, t_embs1, h_embs2, r_embs2, t_embs2 = self.getEmbeddings(heads, rels, tails, years, months, days, alldata)
        scores = ((h_embs1 * r_embs1) * t_embs1 + (h_embs2 * r_embs2) * t_embs2) / 2.0
        scores = F.dropout(scores, p=self.params.dropout, training=self.training)
        scores = torch.sum(scores, dim=1)
        return scores


# DE_simple.py

In [23]:

import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

class DE_SimplE(torch.nn.Module):
    def __init__(self, dataset, params):
        super(DE_SimplE, self).__init__()
        self.dataset = dataset
        self.params = params
        
        self.ent_embs_h = nn.Embedding(dataset.numEnt(), params.s_emb_dim).cuda()
        self.ent_embs_t = nn.Embedding(dataset.numEnt(), params.s_emb_dim).cuda()

        self.rel_embs_f = nn.Embedding(dataset.numRel(), params.s_emb_dim+params.t_emb_dim).cuda()
        self.rel_embs_i = nn.Embedding(dataset.numRel(), params.s_emb_dim+params.t_emb_dim).cuda()
        
        self.create_time_embedds()

        self.time_nl = torch.sin  
        
        nn.init.xavier_uniform_(self.ent_embs_h.weight)
        nn.init.xavier_uniform_(self.ent_embs_t.weight)
        nn.init.xavier_uniform_(self.rel_embs_f.weight)
        nn.init.xavier_uniform_(self.rel_embs_i.weight)
    
    def create_time_embedds(self):

        self.m_freq_h = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.m_freq_t = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.d_freq_h = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.d_freq_t = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.y_freq_h = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.y_freq_t = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()

        self.m_phi_h = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.m_phi_t = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.d_phi_h = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.d_phi_t = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.y_phi_h = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.y_phi_t = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()

        self.m_amps_h = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.m_amps_t = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.d_amps_h = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.d_amps_t = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.y_amps_h = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.y_amps_t = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()

        nn.init.xavier_uniform_(self.m_freq_h.weight)
        nn.init.xavier_uniform_(self.d_freq_h.weight)
        nn.init.xavier_uniform_(self.y_freq_h.weight)
        nn.init.xavier_uniform_(self.m_freq_t.weight)
        nn.init.xavier_uniform_(self.d_freq_t.weight)
        nn.init.xavier_uniform_(self.y_freq_t.weight)

        nn.init.xavier_uniform_(self.m_phi_h.weight)
        nn.init.xavier_uniform_(self.d_phi_h.weight)
        nn.init.xavier_uniform_(self.y_phi_h.weight)
        nn.init.xavier_uniform_(self.m_phi_t.weight)
        nn.init.xavier_uniform_(self.d_phi_t.weight)
        nn.init.xavier_uniform_(self.y_phi_t.weight)

        nn.init.xavier_uniform_(self.m_amps_h.weight)
        nn.init.xavier_uniform_(self.d_amps_h.weight)
        nn.init.xavier_uniform_(self.y_amps_h.weight)
        nn.init.xavier_uniform_(self.m_amps_t.weight)
        nn.init.xavier_uniform_(self.d_amps_t.weight)
        nn.init.xavier_uniform_(self.y_amps_t.weight)

    def get_time_embedd(self, entities, years, months, days, h_or_t):
        if h_or_t == "head":
            emb  = self.y_amps_h(entities) * self.time_nl(self.y_freq_h(entities) * years  + self.y_phi_h(entities))
            emb += self.m_amps_h(entities) * self.time_nl(self.m_freq_h(entities) * months + self.m_phi_h(entities))
            emb += self.d_amps_h(entities) * self.time_nl(self.d_freq_h(entities) * days   + self.d_phi_h(entities))
        else:
            emb  = self.y_amps_t(entities) * self.time_nl(self.y_freq_t(entities) * years  + self.y_phi_t(entities))
            emb += self.m_amps_t(entities) * self.time_nl(self.m_freq_t(entities) * months + self.m_phi_t(entities))
            emb += self.d_amps_t(entities) * self.time_nl(self.d_freq_t(entities) * days   + self.d_phi_t(entities))
            
        return emb

    def getEmbeddings(self, heads, rels, tails, years, months, days, intervals = None):
        years = years.view(-1,1)
        months = months.view(-1,1)
        days = days.view(-1,1)
        h_embs1 = self.ent_embs_h(heads)
        r_embs1 = self.rel_embs_f(rels)
        t_embs1 = self.ent_embs_t(tails)
        h_embs2 = self.ent_embs_h(tails)
        r_embs2 = self.rel_embs_i(rels)
        t_embs2 = self.ent_embs_t(heads)
        
        h_embs1 = torch.cat((h_embs1, self.get_time_embedd(heads, years, months, days, "head")), 1)
        t_embs1 = torch.cat((t_embs1, self.get_time_embedd(tails, years, months, days, "tail")), 1)
        h_embs2 = torch.cat((h_embs2, self.get_time_embedd(tails, years, months, days, "head")), 1)
        t_embs2 = torch.cat((t_embs2, self.get_time_embedd(heads, years, months, days, "tail")), 1)

        return h_embs1, r_embs1, t_embs1, h_embs2, r_embs2, t_embs2
    
    def forward(self, heads, rels, tails, years, months, days,alldata):
        h_embs1, r_embs1, t_embs1, h_embs2, r_embs2, t_embs2 = self.getEmbeddings(heads, rels, tails, years, months, days)
        scores = ((h_embs1 * r_embs1) * t_embs1 + (h_embs2 * r_embs2) * t_embs2) / 2.0
        scores = F.dropout(scores, p=self.params.dropout, training=self.training)
        scores = torch.sum(scores, dim=1)
        return scores
        


# DE_TransE.py

In [24]:


class DE_TransE(torch.nn.Module):
    def __init__(self, dataset, params):
        super(DE_TransE, self).__init__()
        self.dataset = dataset
        self.params = params
        
        self.ent_embs = nn.Embedding(dataset.numEnt(), params.s_emb_dim).cuda()
        self.rel_embs = nn.Embedding(dataset.numRel(), params.s_emb_dim+params.t_emb_dim).cuda()
        
        self.create_time_embedds()
        
        self.time_nl = torch.sin
        
        nn.init.xavier_uniform_(self.ent_embs.weight)
        nn.init.xavier_uniform_(self.rel_embs.weight)
        
        self.sigm = torch.nn.Sigmoid()
        self.tanh = nn.Tanh()
    
    def create_time_embedds(self):
            
        self.m_freq = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.d_freq = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.y_freq = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()

        nn.init.xavier_uniform_(self.m_freq.weight)
        nn.init.xavier_uniform_(self.d_freq.weight)
        nn.init.xavier_uniform_(self.y_freq.weight)

        self.m_phi = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.d_phi = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.y_phi = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()

        nn.init.xavier_uniform_(self.m_phi.weight)
        nn.init.xavier_uniform_(self.d_phi.weight)
        nn.init.xavier_uniform_(self.y_phi.weight)

        self.m_amp = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.d_amp = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()
        self.y_amp = nn.Embedding(self.dataset.numEnt(), self.params.t_emb_dim).cuda()

        nn.init.xavier_uniform_(self.m_amp.weight)
        nn.init.xavier_uniform_(self.d_amp.weight)
        nn.init.xavier_uniform_(self.y_amp.weight)

    def get_time_embedd(self, entities, year, month, day):
        
        y = self.y_amp(entities)*self.time_nl(self.y_freq(entities)*year + self.y_phi(entities))
        m = self.m_amp(entities)*self.time_nl(self.m_freq(entities)*month + self.m_phi(entities))
        d = self.d_amp(entities)*self.time_nl(self.d_freq(entities)*day + self.d_phi(entities))
        
        return y+m+d

    def getEmbeddings(self, heads, rels, tails, years, months, days, intervals = None):
        years = years.view(-1,1)
        months = months.view(-1,1)
        days = days.view(-1,1)

        h,r,t = self.ent_embs(heads), self.rel_embs(rels), self.ent_embs(tails)
        
        h_t = self.get_time_embedd(heads, years, months, days)
        t_t = self.get_time_embedd(tails, years, months, days)
        
        h = torch.cat((h,h_t), 1)
        t = torch.cat((t,t_t), 1)
        return h,r,t
    
    def forward(self, heads, rels, tails, years, months, days,alldata):
        h_embs, r_embs, t_embs = self.getEmbeddings(heads, rels, tails, years, months, days)
        
        scores = h_embs + r_embs - t_embs
        scores = F.dropout(scores, p=self.params.dropout, training=self.training)
        scores = -torch.norm(scores, dim = 1)
        return scores
        

# tester.py

In [25]:
import torch
import numpy as np

class Tester:
    def __init__(self, dataset, model_path, valid_or_test):
        self.model = torch.load(model_path)
        self.model.eval()
        self.dataset = dataset
        self.valid_or_test = valid_or_test
        self.measure = Measure()
        
        self.data_tensor = torch.tensor(self.dataset.data[self.valid_or_test]).to(next(self.model.parameters()).device)
        self.data_tensor = torch.tensor(self.dataset.data["train"]).to(next(self.model.parameters()).device)

    def getRank(self, sim_scores):  
        return (sim_scores > sim_scores[0]).sum() + 1
    
    def replaceAndShred(self, fact, raw_or_fil, head_or_tail):
        head, rel, tail, years, months, days = fact
        num_entities = self.dataset.numEnt()

        if head_or_tail == "head":
            ret_facts = [(i, rel, tail, years, months, days) for i in range(num_entities)]
        else:  # head_or_tail == "tail"
            ret_facts = [(head, rel, i, years, months, days) for i in range(num_entities)]
        
        if raw_or_fil == "raw":
            ret_facts = [fact] + ret_facts
        else:  # raw_or_fil == "fil"
            ret_facts = [fact] + list(set(ret_facts) - self.dataset.all_facts_as_tuples)

        return shredFacts(np.array(ret_facts))
    
    def test(self):

        for i, fact in enumerate(self.dataset.data[self.valid_or_test]):
            settings = ["fil"]

            for raw_or_fil in settings:
                for head_or_tail in ["head", "tail"]:
                    heads, rels, tails, years, months, days = self.replaceAndShred(fact, raw_or_fil, head_or_tail)

                    sim_scores = self.model(heads, rels, tails, years, months, days, self.data_tensor).cpu().data.numpy()
                    
                    rank = self.getRank(sim_scores)
                    self.measure.update(rank, raw_or_fil)

        self.measure.print_()
        print("~~~~~~~~~~~~~")
        self.measure.normalize(len(self.dataset.data[self.valid_or_test]))
        self.measure.print_()
        
        return self.measure.mrr["fil"]


# trainer.py

In [32]:

import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

loss_glo = []
model_path_glo = ""

class Trainer:
    def __init__(self, dataset, params, model_name):
        instance_gen = globals()[model_name]
        self.model_name = model_name
        self.model = nn.DataParallel(instance_gen(dataset=dataset, params=params))

        self.dataset = dataset
        self.params = params
        
    def train(self, early_stop=False):
        self.model.train()
        
        optimizer = torch.optim.Adam(
            self.model.parameters(), 
            lr=self.params.lr, 
            weight_decay=self.params.reg_lambda
        ) 
        loss_f = nn.CrossEntropyLoss()
        
        for epoch in range(1, self.params.ne + 1):
            last_batch = False
            total_loss = 0.0
            start = time.time()

            c_batch = 0
            while not last_batch:
                # c_batch += 1
             
                optimizer.zero_grad()
                
                (heads, rels, tails, years, months, days),alldata = self.dataset.nextBatch(self.params.bsize, neg_ratio=self.params.neg_ratio)
                last_batch = self.dataset.wasLastBatch()

                scores = self.model(heads, rels, tails, years, months, days,alldata.cuda())
               
                ###Added for softmax####
                num_examples = int(heads.shape[0] / (1 + self.params.neg_ratio))
                scores_reshaped = scores.view(num_examples, self.params.neg_ratio+1)
                l = torch.zeros(num_examples).long().cuda()
                loss = loss_f(scores_reshaped, l)
                loss.backward()
                optimizer.step()
                total_loss += loss.cpu().item()
                
            print(time.time() - start)
            
            print("Loss in iteration " + str(epoch) + ": " + str(total_loss) + "(" + self.model_name + "," + self.dataset.name + ")")
            
            if epoch % self.params.save_each == 0:
                self.saveModel(epoch)
            global loss_glo
            loss_glo.append(total_loss)
            
    def saveModel(self, chkpnt):
        print("Saving the model")
        directory = "/kaggle/working/models"
        if not os.path.exists(directory):
            os.makedirs(directory)
        global model_path_glo
        model_path_glo = directory + self.params.str_() + "_" + str(chkpnt) + ".chkpnt"
        
            
        torch.save(self.model, directory + self.params.str_() + "_" + str(chkpnt) + ".chkpnt")
        
    

# main.py

In [1]:

bs = 1024
neg_ratio = 150
se_prop = 0.36
emb_dim = 100
ne = 50
save_each = 50
model_ = 'G_DE_SimplE'  # 'DE_TransE', 'DE_SimplE','G_DE_SimplE'
expDataset = '/kaggle/input/dataknowledgegraph/datasets/icews14/'





class Args:
    def __init__(self):
        self.dataset = expDataset
        self.model = model_
        self.ne = ne
        self.bsize = bs
        self.lr = 0.001
        self.reg_lambda = 0.0
        self.emb_dim = emb_dim
        self.neg_ratio = neg_ratio
        self.dropout = 0.4
        self.save_each = save_each
        self.se_prop = se_prop


args = Args()
dataset = Dataset(args.dataset)
params = Params(
    ne=args.ne, 
    bsize=args.bsize, 
    lr=args.lr, 
    reg_lambda=args.reg_lambda, 
    emb_dim=args.emb_dim, 
    neg_ratio=args.neg_ratio, 
    dropout=args.dropout, 
    save_each=args.save_each, 
    se_prop=args.se_prop
)


print("Start training")
trainer = Trainer(dataset, params, args.model)
trainer.train()

print("start testing")
model_path = model_path_glo
tester = Tester(dataset, model_path, "test")
tester.test()



# Save inf model
def printInf():

    print("Hyperparameter: ")
    print("Number epoch (ne):", params.ne)
    print("Batch Size (bsize):", params.bsize)

    print("Embedding Dimension (emb_dim):", params.emb_dim)
    print("Negative Ratio (neg_ratio):", params.neg_ratio)

    print("SE Property (se_prop):", params.se_prop)
    print("")
    
    print("Loss:")
    print(loss_glo)
    print("")

    print("Test")
    tester.measure.print__()
printInf()

file_name = f"ne_{params.ne}_bsize_{params.bsize}_embdim_{params.emb_dim}_neg_{params.neg_ratio}_seprop_{params.se_prop}.txt"
file_path = f"/kaggle/working/exp/{file_name}"


os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, 'w') as f:
    with contextlib.redirect_stdout(f):
        printInf()
